In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import h5py
import numpy as np

h5f= h5py.File('/content/drive/My Drive/googlecolab/SVHN_single_grey1.h5', 'r')

In [0]:
X_train=h5f['X_train'][:4000]
X_test=h5f['X_test'][:1000]
Y_train=h5f['y_train'][:4000]
Y_test=h5f['y_test'][:1000]

In [0]:
rows, x, y =X_train.shape
X_train1=np.reshape(X_train, (rows,x*y))

In [0]:

trows, tx, ty =X_test.shape
X_test1=np.reshape(X_test, (trows,tx*ty))

In [0]:
print(X_train1.shape)
print(X_test1.shape)

(4000, 1024)
(1000, 1024)


In [0]:
from sklearn.model_selection import train_test_split
X_tr, X_va, Y_tr, Y_va =train_test_split(X_train1,Y_train, test_size=0.2, random_state=7)

In [0]:
# Load libraries
from matplotlib import pyplot
import pandas as pd
from pandas import set_option
import seaborn as sn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
%matplotlib inline
from scipy.stats import zscore  

In [0]:
# Test options and evaluation metric

num_folds = 5
seed = 7
scoring = 'mean_squared_error'
model = KNeighborsClassifier()    

In [0]:
# Tune Hyper parameter 
scaler = StandardScaler().fit(X_tr)
rescaledX = scaler.transform(X_tr)
k_values = np.array([1,3,5,7,9,11,13,15,20,30,40,50])
param_grid = dict(n_neighbors=k_values)
kfold = KFold(n_splits=num_folds, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=kfold)
grid_result = grid.fit(rescaledX, Y_tr)

In [0]:
print("Best: %f using %s" % (-grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (-mean, stdev, param))

Best: -0.316250 using {'n_neighbors': 13}
-0.299687 (0.014300) with: {'n_neighbors': 1}
-0.282813 (0.018957) with: {'n_neighbors': 3}
-0.305312 (0.026007) with: {'n_neighbors': 5}
-0.313437 (0.019101) with: {'n_neighbors': 7}
-0.309063 (0.014604) with: {'n_neighbors': 9}
-0.312500 (0.015842) with: {'n_neighbors': 11}
-0.316250 (0.014974) with: {'n_neighbors': 13}
-0.312812 (0.015793) with: {'n_neighbors': 15}
-0.310000 (0.025012) with: {'n_neighbors': 20}
-0.292500 (0.017128) with: {'n_neighbors': 30}
-0.293750 (0.024287) with: {'n_neighbors': 40}
-0.283438 (0.023812) with: {'n_neighbors': 50}


In [0]:
model2 = KNeighborsClassifier (n_neighbors=13)   
model2.fit(scaler.transform(X_tr),Y_tr)
model2.score(scaler.transform(X_tr),Y_tr)

0.4940625

In [0]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix

prediction=model2.predict(scaler.transform(X_va))
print("Accuracy:",metrics.accuracy_score(prediction,Y_va))

Accuracy: 0.33


In [0]:
prediction=model2.predict(scaler.transform(X_test1))
print("Accuracy:",metrics.accuracy_score(prediction,Y_test))

Accuracy: 0.373


In [0]:
# Confusion Metrix 
print("Confusion Metrix:   \n", metrics.confusion_matrix(prediction,Y_test))

Confusion Metrix:   
 [[68 13 12  9  8 21 26  9 17 29]
 [11 56 13 16 18 13  3 16  3 11]
 [ 0  2 32 10  3  3  1 11  5  5]
 [ 2  7  6 27  6 13  7  9  4  7]
 [ 8  7  5  4 42  3 14  2 10  6]
 [ 1  1  1  8  2 21  5  3  6  5]
 [ 8  2  4  4  4 14 26  2 15  5]
 [ 4  2  7  3  5  3  1 48  2  7]
 [ 5  3  5  2  2  6  4  1 26  9]
 [10  0  7  2  6  8  7  6 12 27]]


In [0]:
metrics.f1_score(Y_test, prediction,average=None)

array([0.41337386, 0.44268775, 0.3902439 , 0.31213873, 0.42639594,
       0.26582278, 0.29213483, 0.50793651, 0.3190184 , 0.2755102 ])

In [0]:
metrics.precision_score(Y_test, prediction,average=None)

array([0.32075472, 0.35      , 0.44444444, 0.30681818, 0.41584158,
       0.39622642, 0.30952381, 0.58536585, 0.41269841, 0.31764706])

In [0]:
metrics.recall_score(Y_test, prediction,average=None)

array([0.58119658, 0.60215054, 0.34782609, 0.31764706, 0.4375    ,
       0.2       , 0.27659574, 0.44859813, 0.26      , 0.24324324])

In [0]:
import tensorflow as tf
tf.keras.backend.clear_session()
tf.set_random_seed(42)

In [0]:
X_train=h5f['X_train'][:4000]
X_test=h5f['X_test'][:1000]
Y_train=h5f['y_train'][:4000]
Y_test=h5f['y_test'][:1000]

Y_train1 = tf.keras.utils.to_categorical(Y_train, num_classes=10)
Y_test1 = tf.keras.utils.to_categorical(Y_test, num_classes=10)
print(X_train.shape)
print(X_test.shape)
print(Y_train1.shape)
print(Y_test1.shape)

(4000, 32, 32)
(1000, 32, 32)
(4000, 10)
(1000, 10)


In [0]:
model = tf.keras.models.Sequential()

#Reshape data from 2D (28,28) to 1D (784, )
model.add(tf.keras.layers.Reshape((1024,),input_shape=(32,32,)))

#normalize data
model.add(tf.keras.layers.BatchNormalization())

In [0]:
#Dense layer 1

model.add(tf.keras.layers.Dense(200, activation='relu'))

#Dense layer 2
model.add(tf.keras.layers.Dense(100, activation='relu'))

# Add dropout
model.add(tf.keras.layers.Dropout(0.25))

In [0]:
#output layer 
model.add(tf.keras.layers.Dense(10, activation='softmax'))

#Compile

sgd_optimizer = tf.keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)

model.compile(sgd_optimizer, 
              loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 1024)              0         
_________________________________________________________________
batch_normalization_v1 (Batc (None, 1024)              4096      
_________________________________________________________________
dense (Dense)                (None, 200)               205000    
_________________________________________________________________
dense_1 (Dense)              (None, 100)               20100     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 230,206
Trainable params: 228,158
Non-trainable params: 2,048
_________________________________________________________________


In [0]:
#Train the model

earlyStopping=tf.keras.callbacks.EarlyStopping(patience=5)


model.fit(X_train,Y_train1,          
          validation_data=(X_test,Y_test1),
          epochs=10,
          batch_size=32,
         callbacks=[earlyStopping])

Train on 4000 samples, validate on 1000 samples
Epoch 1/10
4000/4000 [==============================] - 1s 150us/sample - loss: 2.3954 - acc: 0.1357 - val_loss: 2.1958 - val_acc: 0.2320
Epoch 2/10
4000/4000 [==============================] - 0s 112us/sample - loss: 2.1527 - acc: 0.2305 - val_loss: 2.0121 - val_acc: 0.3120
Epoch 3/10
4000/4000 [==============================] - 0s 110us/sample - loss: 1.9809 - acc: 0.2990 - val_loss: 1.8447 - val_acc: 0.4160
Epoch 4/10
4000/4000 [==============================] - 0s 112us/sample - loss: 1.8272 - acc: 0.3790 - val_loss: 1.6782 - val_acc: 0.4830
Epoch 5/10
4000/4000 [==============================] - 0s 107us/sample - loss: 1.6972 - acc: 0.4397 - val_loss: 1.5586 - val_acc: 0.5250
Epoch 6/10
4000/4000 [==============================] - 0s 106us/sample - loss: 1.5735 - acc: 0.4823 - val_loss: 1.4358 - val_acc: 0.5740
Epoch 7/10
4000/4000 [==============================] - 0s 107us/sample - loss: 1.4966 - acc: 0.5027 - val_loss: 1.3361 - va

In [0]:
prediction1=model.predict_classes(X_test)

In [0]:
metrics.accuracy_score(prediction1,Y_test)

0.65

In [0]:
metrics.f1_score(Y_test, prediction1,average=None)

array([0.65612648, 0.65838509, 0.71264368, 0.55276382, 0.69767442,
       0.60301508, 0.46067416, 0.79069767, 0.65306122, 0.68571429])

In [0]:
metrics.precision_score(Y_test, prediction1,average=None)

array([0.61029412, 0.77941176, 0.75609756, 0.48245614, 0.6302521 ,
       0.63829787, 0.48809524, 0.78703704, 0.66666667, 0.72727273])

In [0]:
metrics.recall_score(Y_test, prediction1,average=None)

array([0.70940171, 0.56989247, 0.67391304, 0.64705882, 0.78125   ,
       0.57142857, 0.43617021, 0.79439252, 0.64      , 0.64864865])

In [0]:
The DNN has way better performance than KNN model in test data